In [ ]:
import matplotlib.font_manager as font_manager
from matplotlib.lines import Line2D
import matplotlib as mpl
import matplotlib.ticker as mtick
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
params = {
    "axes.titlesize" : 14,
    'axes.labelsize': 12,
    'font.size': 12,
    'legend.fontsize': 12,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'text.usetex': False
}

mpl.rcParams.update(params)


import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_time_series = pd.read_csv("./data/df_timeseries_en.tsv.gz", compression="infer", sep="\t")
df_channels = pd.read_csv("./data/df_channels_en.tsv.gz", compression="infer", sep="\t")
df_time_series["datetime"] = pd.to_datetime(df_time_series["datetime"])
df_channels["join_date"] = pd.to_datetime(df_channels["join_date"])
video_metadatas = pd.read_feather("./data/yt_metadata_helper.feather", columns=["duration", "like_count", "view_count", "channel_id"])
video_metadatas["dummmy"] = 1

# Check length of videos

An idea would be to use cut to match length of videos into discrete intervals 

In [ ]:
bucket_durations = pd.cut(video_metadatas['duration'], bins=100)
print(type(bucket_durations))

---

# Check frequency of videos

---

# How long to reach 100K subscribers for ex 

Could be interesting to take all the videos with 10K, 20K, 30K, ... and see the evolution to reach 100K for example. And compare different evolution : from 50K to 100K or from 500K to 600K, etc.. which one is the fastest, easiest ?  

In [48]:
under_10K = df_time_series[df_time_series['subs'] < 10_000]['channel']
more_10K = df_time_series[df_time_series['subs'] > 100_000]['channel']
print(under_10K.isin(more_10K).value_counts())
print('We have {} channels that begins with 10K and reach at least 100K'.format(under_10K.isin(more_10K).sum()))

False    4769814
True      463833
Name: channel, dtype: int64
We have 463833 channels that begins with 10K and reach at least 100K


Good new, we have a lot of data to analyze!

---

In [ ]:
video_metadatas.sample(7)

In [ ]:
video_metadatas['duration'].describe()

In [ ]:
print(df_time_series['category'].unique())
print(len(df_time_series['category']))
df_time_series.head(10)

In [ ]:
print(df_channels['category_cc'].unique())
print(len(df_channels['category_cc']))
df_channels.head(7)

In [ ]:
# print the number of channels created every year 
channel_year = df_channels['join_date'].apply(lambda d: d.to_pydatetime().year)
# channel_year.value_counts().sort_values().plot(kind='bar')

In [ ]:
top_ranked_channels = df_channels[df_channels.subscriber_rank_sb < 100]
top_ranked_channels.head()

In [ ]:
date = top_ranked_channels["join_date"].apply(lambda d: d.to_pydatetime().year)
np.mean(date, axis=0)

In [ ]:
oldest_video = df_channels['join_date'].apply(lambda d: d.to_pydatetime().year)
oldest_video.nsmallest(5)

In [ ]:
oldest_from_series = df_time_series['datetime'].apply(lambda d: d.to_pydatetime().year)
oldest_from_series.nsmallest(4)

In [ ]:
channels_with_largest_subscribers = df_channels.nlargest(53, 'subscribers_cc')
channels_with_largest_subscribers.sample(7)

In [ ]:
df_time_series.sample(7)

In [ ]:
df_time_series[df_time_series['channel'] == 'UC-lHJZR3Gqxm24_Vd_AJ5Yw'].head()